<a href="https://colab.research.google.com/github/wallybeamm/neural_entity_recognition/blob/feature%2Finit/enitity_recognation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entity Recognotion

## Academic Background

## Practical Implementations

Set parameters

In [ ]:
json_path = '/content/test_data.jsonl'
model_name = 'en_trf_bertbaseuncased_lg-'

output_dir = "/content/Model"
n_iter = 100

Import librariers

In [ ]:
import json
import spacy 
import random
from spacy.training.example import Example

!pip3 install spacy-transformers
!python -m spacy download en_trf_bertbasecased_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-30 00:11:05.392257: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

✘ No compatible package found for 'en_trf_bertbasecased_lg' (spaCy
v3.4.1)



In [ ]:
# Load the dataset

def load_dataset(path):
  data = []
  for line in open(path, 'r'):
      line_dict = json.loads(line)
      data.append((line_dict['data'].replace('\n', ' '), line_dict['label']))
  return data

# Display entity info
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
  else: 
    print('No named entities found.')

def train(data, model):
  # Update the label list
  for annotations in data:
      for ent in annotations[1]:
          model.add_label(ent[2])
          

  other_pipes = [pipe for pipe in model.pipe_names if pipe != 'ner']
  with model.disable_pipes(*other_pipes):  # only train NER
      optimizer = model.begin_training()
      for itn in range(n_iter):
          random.shuffle(train_data)
          losses = {}
          batches = spacy.util.minibatch(train_data, size=2)
          for batch in batches:
              for text, annotations in batch:
                  # create Example
                  doc = model.make_doc(text)
                  annotations = {'entities' : annotations}
                  example = Example.from_dict(doc, annotations)
                  # Update the model
                  model.update([example], losses=losses, drop=0.3)
  return model

def split(data, train_percantage):
  # Split the data
  train_lenght = int(len(data)*train_percantage)
  train_data = data[:train_lenght]
  test_data = data[train_lenght:]
  return train_data, test_data

def test(test_data, model):
  for text, _ in test_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

def same_model(model, output_dir):
  if output_dir is not None:
      nlp.to_disk(output_dir)
      print("Saved model to", output_dir)


In [ ]:
# Main
data = load_dataset(json_path)

nlp = spacy.load(model_name)

train_data, test_data = split(data, 0.7)

finetuned_model = train(train_data, nlp)

OSError: ignored

In [ ]:
if output_dir is not None:
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/Model
